<a href="https://colab.research.google.com/github/mehlmanmichael/Coursera_Capstone/blob/main/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction / Business Problem

* I want to know what neighborhood in Houston would be good to buy a house in.  
* First cluster.  
    * Get foursquare data.  
    * As well as school data and other socioeconomic data if you can find it.
        * if you can find socioeconomic data, make sure you'r weighting it correctly so that the foursquare data doesn't dominate the fit.    
    * Do not include housing price data.  
    * Find similar neighborhoods via clustering (probably use DBSCAN to avoid outliers).  
* Then for each cluster look at the neighborhoods that are outliers in terms of house price (undepriced).  Probably just do this with some type of boxplot.
    * Use zillo api or similar to find average houst price in a neighborhood

## Required data
* List of houston *zip* codes or neighborhoods with lat/lon
* Would be nice to have a geojson file to draw neighborhood boundaries
* List of socioeconomic data for neighborhoods (salary, school rating, crime etc.)
* List of home prices

# Import libraries and load API keys

In [153]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
import folium # plotting library
from folium.features import DivIcon
import json
from sklearn.cluster import KMeans
#from bs4 import BeautifulSoup
import json
#from sklearn.cluster import DBSCAN 
import sklearn.utils
#!pip install pyzillow
#import pyzillow
import matplotlib.cm as cm
import matplotlib.colors as colors

In [151]:
# Foursquare API credentials are stored on my google drive in a .py file.  Load them here.
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
libpath = '/content/drive/My Drive/Programming/Python/Coursera Data Science Capstone/'
sys.path.append(libpath)
import keys
drive.flush_and_unmount()
#print(keys.CLIENT_ID+keys.CLIENT_SECRET+keys.ACCESS_TOKEN+keys.ZWSID)
VERSION = '20180604'
LIMIT = 50

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get Houston neighborhoods and lat/lon

In [4]:
geo_json_data = json.loads(requests.get('https://mycity2.houstontx.gov/pubgis02/rest/services/HoustonMap/Administrative_Bourndary/MapServer/3?f=pjson').text)

In [26]:
geolocator = Nominatim(user_agent='explorer')
location = geolocator.geocode('Houston, Texas')
hlat = location.latitude
hlon = location.longitude
regs = ['770{:02d}'.format(i) for i in range(100)]
locs = [geolocator.geocode(reg+', Texas') for reg in regs]
lats = [loc.latitude if loc is not None else None for loc in locs]
lons = [loc.longitude if loc is not None else None for loc in locs]

In [125]:
lat1, lon1 = 29.828408887699396, -95.38141035479127
lat2, lon2 = 29.703835072319947, -95.51242592009352
latlist = np.linspace(lat1, lat2, 10)
lonlist = np.linspace(lon1, lon2, 10)
lats, lons, regs = [], [], []
i = 0
for lat in latlist:
  for lon in lonlist:
    lats.append(lat)
    lons.append(lon)
    regs.append(i)
    i += 1
#map = folium.Map(location=[hlat, hlon], zoom_start=12)
#for lat, lon in zip(lats,lons):
#  folium.CircleMarker([lat, lon], radius=5, fill=True).add_to(map)    
#map

In [128]:
df = pd.DataFrame({'reg':regs,'lat':lats,'lon':lons})
df = df.dropna().set_index('reg')
df.head()

,lat,lon
reg,,
0,29.828409,-95.381410
1,29.828409,-95.395968
2,29.828409,-95.410525
3,29.828409,-95.425082
4,29.828409,-95.439639


In [129]:
map = folium.Map(location=[hlat, hlon], zoom_start=12)
for lat, lon, in zip(df.lat, df.lon):
    folium.CircleMarker([lat, lon], radius=5, fill=True).add_to(map)
map

In [130]:
LIMIT = 50
def getNearbyVenues(regs, lats, lons, radius=2000):
    venues_list=[]
    for zc, lat, lon in zip(regs, lats, lons):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(keys.CLIENT_ID, keys.CLIENT_SECRET, VERSION, lat, lon, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(zc, lat, lon, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['reg', 'lat', 'lon', 'venue', 'venue_lat', 'venue_lon', 'venue_category']
    return(nearby_venues)

In [131]:
venues = getNearbyVenues(regs=df.index.tolist(), lats=df['lat'], lons=df['lon'])

In [132]:
onehot = pd.get_dummies(venues[['venue_category']], prefix="", prefix_sep="")
onehot['reg'] = venues['reg'] 
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
grouped = onehot.groupby('reg').mean().reset_index()
grouped.head()

,reg,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Cafeteria,...,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.04,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.02,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.02,0.00,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.02,0.0,0.0,0.0,0.04,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.04,0.02,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.00,0.0,0.0,0.0,0.06,0.0,0.0,...,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.02,0.0,0.00,0.0,0.0,0.02,0.0,0.02,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.04,0.02,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.00,0.0,0.0,0.0,0.00,0.0,0.0,...,0.02,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.02,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.04,0.04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.02,0.00,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.02,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.02,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0


In [133]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [147]:
num_top_venues = 10
n_clusters = 10
columns = ['reg']
for ind in np.arange(num_top_venues):
    columns.append('{}_most_common_venue'.format(ind+1))
regs_venues_sorted = pd.DataFrame(columns=columns)
regs_venues_sorted['reg'] = grouped['reg']
for ind in np.arange(grouped.shape[0]):
    regs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)
grouped_clustering = grouped.drop('reg', 1)
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(grouped_clustering)#fit clusters
regs_venues_sorted.insert(0, 'cluster_label', kmeans.labels_)
#db = DBSCAN(eps=0.001, min_samples=500).fit(grouped_clustering)
#core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#core_samples_mask[db.core_sample_indices_] = True
#labels = db.labels_
#regs_venues_sorted.insert(0, 'cluster_label', db.labels_)

In [148]:
merged = df
merged = merged.join(regs_venues_sorted.set_index('reg'))
print(merged['cluster_label'].unique())
merged

[2 3 6 7 4 0 8 9 5 1]


,lat,lon,cluster_label,1_most_common_venue,2_most_common_venue,3_most_common_venue,4_most_common_venue,5_most_common_venue,6_most_common_venue,7_most_common_venue,8_most_common_venue,9_most_common_venue,10_most_common_venue
reg,,,,,,,,,,,,,
0,29.828409,-95.381410,2,Mobile Phone Shop,Discount Store,BBQ Joint,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Bank,Pharmacy,Ice Cream Shop,Sandwich Place
1,29.828409,-95.395968,2,Mexican Restaurant,Discount Store,Department Store,Burger Joint,Pizza Place,Mobile Phone Shop,BBQ Joint,Beer Garden,Clothing Store,Fried Chicken Joint
2,29.828409,-95.410525,3,Coffee Shop,Burger Joint,Pizza Place,Gym,Brewery,Bank,Discount Store,BBQ Joint,Beer Garden,Mexican Restaurant
3,29.828409,-95.425082,3,Pizza Place,Food Truck,Gym,Mexican Restaurant,Beer Garden,Pet Store,Park,Coffee Shop,Bank,Gas Station
4,29.828409,-95.439639,2,Park,Pizza Place,Gym,Sandwich Place,Salon / Barbershop,Bar,Chinese Restaurant,Bank,Discount Store,Food Truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29.703835,-95.454197,3,Pizza Place,Coffee Shop,American Restaurant,Grocery Store,Burger Joint,Sandwich Place,Restaurant,Park,Gym,Bakery
96,29.703835,-95.468754,3,Pizza Place,Coffee Shop,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Grocery Store,Burger Joint,Restaurant,Breakfast Spot,Sushi Restaurant
97,29.703835,-95.483311,3,Mexican Restaurant,Pizza Place,Coffee Shop,Latin American Restaurant,Taco Place,Spanish Restaurant,Ice Cream Shop,Grocery Store,Seafood Restaurant,Gym


In [149]:
map_clusters = folium.Map(location=[hlat, hlon], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, poi, cluster in zip(merged['lat'], merged['lon'], merged.index.tolist(), merged['cluster_label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
    folium.map.Marker([lat, lon], icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),html='<div style="font-size: 24pt">%s</div>' % str(cluster))).add_to(map_clusters)
    folium.Circle([lat, lon], 500, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [154]:
from pyzillow.pyzillow import ZillowWrapper
zillow_data = ZillowWrapper(keys.ZWSID)

In [160]:
deep_search_response = zillow_data.get_deep_search_results('2114 Bigelow Ave', '98109', True)
result = GetDeepSearchResults(deep_search_response)
result

ZillowError: ignored

In [158]:
url = 'https://api.gateway.attomdata.com/propertyapi/v1.0.0/property/snapshot?latitude=39.7047&longitude=-105.0814&radius=2'
results = requests.get(url).json()
print(results)

{'Response': {'status': {'version': '1.0.0', 'code': '401', 'msg': 'Unauthorized', 'total': '0', 'page': '0', 'pagesize': '0'}}}


In [ ]:
#do this over all businesses!
cshops = onehot.groupby('reg').sum()
cshops = cshops.join(regs_venues_sorted)#.set_index('reg'))
cshops = cshops[['cluster_label', 'Coffee Shop']]
cshops_mean = cshops.groupby('cluster_label').mean()
cshops['reg'] = cshops.index
cshops = cshops.merge(cshops_mean, on='cluster_label')
cshops['pct_sat'] = -100*(cshops['Coffee Shop_x'] - cshops['Coffee Shop_y'])/cshops['Coffee Shop_y'] #mult by -1 _X is reality, #_Y is mean, so a a negative % is oversaturated and a positive number is undersaturated
cshops = cshops[['cluster_label', 'pct_sat', 'reg']].set_index('reg')
cshops['norm_pct'] = (2*(cshops['pct_sat']-cshops['pct_sat'].min())/(cshops['pct_sat'].max() - cshops['pct_sat'].min()))-1
ax = cshops['norm_pct'].plot.hist(bins=10)
cshops = merged[['lat', 'lon']].join(cshops)
cshops.sort_values('norm_pct', ascending=False).head(20)

In [233]:
v_rep = onehot.groupby('reg').sum()
cols = v_rep.columns
v_rep = regs_venues_sorted[['reg','cluster_label']].join(merged[['lat', 'lon']].join(v_rep))
v_rep_mean = v_rep.groupby('cluster_label').mean().iloc[:,3:]
v_rep = v_rep.merge(v_rep_mean, on='cluster_label', suffixes=['', '_avg']).set_index('reg')
for col in cols:
  v_rep[col] = -100*(v_rep[col] - v_rep[col+'_avg'])/v_rep[col+'_avg']
v_rep = v_rep.drop([x+'_avg' for x in cols], axis=1).fillna(0)

,cluster_label,lat,lon,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,...,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
reg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,29.828409,-95.381410,0.0,0.0,0.0,100.0,100.0,0.0,-87.5,100.0,0.0,0.0,0.0,0.0,100.0,-462.5,0.0,100.0,-42.857143,100.00,100.0,100.0,100.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,0.0,-66.666667,100.0,100.0,0.0,100.0,0.0,16.666667,...,100.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0,100.0,-114.285714,-275.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,100.0,0.0,0.0,0.0,0.0,100.0,100.0,100.00,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,0.0
1,2,29.828409,-95.395968,0.0,0.0,0.0,100.0,-400.0,0.0,-87.5,100.0,0.0,0.0,0.0,0.0,100.0,-275.0,0.0,100.0,28.571429,100.00,100.0,100.0,-1400.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,0.0,-66.666667,-400.0,-1400.0,0.0,100.0,0.0,-66.666667,...,100.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0,100.0,-114.285714,-275.0,100.0,100.0,0.0,-25.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,100.0,0.0,0.0,0.0,0.0,100.0,100.0,100.00,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,0.0
4,2,29.828409,-95.439639,0.0,0.0,0.0,100.0,100.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0,-87.5,0.0,100.0,-42.857143,-87.50,100.0,100.0,100.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,0.0,100.000000,-400.0,100.0,0.0,100.0,0.0,16.666667,...,-1400.0,0.0,0.0,100.0,-200.0,0.0,0.0,100.0,100.0,-114.285714,-275.0,100.0,-1400.0,0.0,-25.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,100.0,0.0,0.0,0.0,0.0,-1400.0,100.0,6.25,100.0,0.0,-650.0,0.0,100.0,0.0,100.0,0.0,0.0
5,2,29.828409,-95.454197,0.0,0.0,0.0,100.0,100.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,100.0,28.571429,-87.50,100.0,100.0,100.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-400.0,0.0,-66.666667,100.0,100.0,0.0,100.0,0.0,16.666667,...,100.0,0.0,0.0,-87.5,100.0,0.0,0.0,100.0,-400.0,100.000000,-275.0,100.0,100.0,0.0,-25.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,100.0,0.0,0.0,0.0,0.0,100.0,-275.0,6.25,100.0,0.0,100.0,0.0,-87.5,0.0,100.0,0.0,0.0
6,2,29.828409,-95.468754,0.0,0.0,0.0,100.0,100.0,0.0,-87.5,100.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,100.0,28.571429,6.25,100.0,100.0,100.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-400.0,0.0,-66.666667,100.0,100.0,0.0,100.0,0.0,-150.000000,...,100.0,0.0,0.0,-87.5,100.0,0.0,0.0,100.0,-400.0,100.000000,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.000000,100.000000,0.0,100.0,0.0,0.0,0.0,0.0,100.0,-275.0,6.25,100.0,0.0,100.0,0.0,-87.5,0.0,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,29.717677,-95.483311,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,-0.0,0.0,100.0,100.000000,0.00,0.0,0.0,0.0,0.0,100.0,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,100.0,-50.000000,...,100.0,100.0,0.0,0.0,-400.0,0.0,0.0,0.0

In [246]:
poi = 'Coffee Shop'
map_clusters = folium.Map(location=[hlat, hlon], zoom_start=11)
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, cluster, val in zip(v_rep['lat'], v_rep['lon'], v_rep['cluster_label'], v_rep[poi]):
    out = str(int(val))+'%'
    folium.map.Marker([lat, lon], icon=DivIcon(icon_size=(150,36),icon_anchor=(20,15),html='<div style="font-size: 18pt">%s</div>' % out)).add_to(map_clusters)
    folium.Circle([lat, lon], 500, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters